# Import packages

In [ ]:

from javascript import require, On, Once, AsyncTask, once, off
import math
import os
import json
from actions import Actions
# Set the timeout to 100,000 milliseconds or disable the timeout
# os.environ['REQ_TIMEOUT'] = '10000000'
os.environ['REQ_TIMEOUT'] = '0'

# Import the javascript libraries for Minecraft
mineflayer = require("mineflayer")
pathfinder = require('mineflayer-pathfinder')
collectBlockPlugin = require('mineflayer-collectblock')
blockFinderPlugin = require('mineflayer-blockfinder')
vec3 = require("vec3")

### Global bot parameters

In [ ]:
# edit the serverPort based on your actual situation.
serverPort = 50000 
serverHost = "localhost"
reconnect = True

In [ ]:
class Agent:
    def __init__(self, playerUsername, botName, serverHost, serverPort, reconnect=True):
        self.playerUsername = playerUsername
        self.botName = botName
        self.serverHost = serverHost
        self.serverPort = serverPort
        self.reconnect = reconnect
        self.bot = None
        self.botArgs = {
            "host": serverHost,
            "port": serverPort,
            "username": botName,
            "hideErrors": False,
        }
        # edit them with paths for hash files provided in your machine.
        self.hash_path = {
            "option": "option_hash.json", 
            "registry": "blueprint_itemsByName.json"
        }
        self.createBot()
        

    def createBot(self):
        self.bot = mineflayer.createBot(self.botArgs)
        self.bot.loadPlugin(pathfinder.pathfinder)
        self.bot.loadPlugin(collectBlockPlugin.plugin)
        self.bot.loadPlugin(blockFinderPlugin)
        self.mcData = require('minecraft-data')(self.bot.version)
        self.movements = pathfinder.Movements(self.bot, self.mcData)
        self.bot.pathfinder.setMovements(self.movements)
        self.actions = Actions(self.bot, self.mcData, self.movements,self.playerUsername)
        self.startEvents()
        #######
        # default for our provided environment map; 
        # you can edit it based on your environment map to specify a pivot position.
        self.pivot_pos = (1, 1, 1) 
        self.option_hash = self.load_option_hash()
        self.registry_hash = self.load_registry_hash()   
        return self.bot
    
    def log(self, message):
        print(f"[{self.bot.username}] {message}")

    def startEvents(self):
        bot = self.bot
        botName = self.botName
        reconnectState = {"reconnect": self.reconnect}

        @On(bot, "login")
        def handleLogin(this):

            botSocket = bot._client.socket
            print(f"[{botName}] Logged in to {botSocket.server if botSocket.server else botSocket._host }")

        @On(bot, "kicked")
        def handleKicked(this, reason, loggedIn):
            if loggedIn:
                print(f"[{botName}] Kicked whilst trying to connect: {reason}")

        @On(bot, "messagestr")
        def handleMessagestr(this, message, messagePosition, jsonMsg, sender, verified=None):
            if messagePosition == "chat" and "quit" in message:
                reconnectState["reconnect"] = False
                this.quit()
            elif messagePosition == "chat" and'come' in message:
                localPlayers = bot.players
                for player in localPlayers:
                        playerData = localPlayers[player]
                        if playerData["uuid"] == sender:
                            target = localPlayers[player].entity
                            
                if not target:
                    bot.chat("I don't see you !")
                    return
                pos = target.position
                bot.pathfinder.setMovements(self.movements)
                bot.pathfinder.setGoal(pathfinder.goals.GoalNear(pos.x, pos.y, pos.z, 1))
            

        @On(bot, "end")
        def handleEnd(this, reason):
            print(f"[{botName}] Disconnected: {reason}")
            # Turn off old events
            off(bot, "login", handleLogin)
            off(bot, "kicked", handleKicked)
            off(bot, "messagestr", handleMessagestr)

            # Reconnect if the reconnect flag is set to True
            if reconnectState["reconnect"]:
                print(f"[{botName}] Attempting to reconnect")
                self.createBot()

            # Last event listener
            off(bot, "end", handleEnd)
        

    def init_env(self):
        '''
        init environment based on the provided map.
        you can edit the "/fill" command with your specific settings and environment.
        '''
        self.bot.chat("/setworldspawn 0 1 0")
        self.bot.chat("/fill 0 0 0 180 0 180 minecraft:barrier")
        self.bot.chat("/fill 0 0 0 -180 0 180 minecraft:barrier")
        self.bot.chat("/fill 0 0 0 180 0 -180 minecraft:barrier")
        self.bot.chat("/fill 0 0 0 -180 0 -180 minecraft:barrier")
        self.bot.chat("/gamerule doDaylightCycle false")
        self.bot.chat("/time set day")
        self.bot.chat("/gamerule sendCommandFeedback false")

    def load_option_hash(self):
        with open(self.hash_path['option'], 'r') as f1:
            option_hash = json.load(f1)
        return option_hash

    def load_registry_hash(self):
        with open(self.hash_path['registry'], 'r') as f1:
            registry_hash = json.load(f1)
        return registry_hash

    def build_by_blueprint(self, blueprint, reverse_block_materials):
        for h in range(len(blueprint)): # y axis
            for d in range(len(blueprint[h])): # z axis
                for w in range(len(blueprint[h][d])): # x axis
                    # adaptation to several MLLM output formats. please improve it according to the actual situation
                    if blueprint[h][d][w] in reverse_block_materials and reverse_block_materials[blueprint[h][d][w]] != 'air' and reverse_block_materials[blueprint[h][d][w]] != 'Air':
                        if '(' in reverse_block_materials[blueprint[h][d][w]]:
                            # print('yes')
                            stairs_name = reverse_block_materials[blueprint[h][d][w]].split('(')[0].strip(' ')
                            orientation = reverse_block_materials[blueprint[h][d][w]].split('(')[1].split(')')[0]
                            flag = self.option_hash[orientation.replace(', ', ',')]
                            if '_' in stairs_name or stairs_name.islower():
                                self.bot.chat(f'/setblock {self.pivot_pos[0]+w} {self.pivot_pos[1]+h} {self.pivot_pos[2]+d} minecraft:{stairs_name}{flag}')
                            else:
                                self.bot.chat(f'/setblock {self.pivot_pos[0]+w} {self.pivot_pos[1]+h} {self.pivot_pos[2]+d} minecraft:{self.registry_hash[stairs_name]}{flag}')
                        else:
                            if '_' in reverse_block_materials[blueprint[h][d][w]] or reverse_block_materials[blueprint[h][d][w]].islower():
                                self.bot.chat(f'/setblock {self.pivot_pos[0]+w} {self.pivot_pos[1]+h} {self.pivot_pos[2]+d} minecraft:{reverse_block_materials[blueprint[h][d][w]]}')
                            else:
                                self.bot.chat(f'/setblock {self.pivot_pos[0]+w} {self.pivot_pos[1]+h} {self.pivot_pos[2]+d} minecraft:{self.registry_hash[reverse_block_materials[blueprint[h][d][w]]]}')

        # print('finished!')


### Set your agents (building agent and observing agent)

In [ ]:
# format: Agent(playerUsername, botName, ...)
agent1 = Agent("forture123123","Agent", "localhost", serverPort)
agent2 = Agent("jessica030327", "bot", "localhost", serverPort)


###
# If you wanna use the visualization tool 'Mineflayer Viewer', just uncomment the following codes, and comment the above 'agent2=Agent()' code line.
# You need to install canvas and corresponding requirements.
###

# canvas = require('canvas')
# viewer = require('prismarine-viewer').mineflayer
# puppeteer = require('puppeteer')
# serverPortViewer = 3100
# new_viewer = viewer(agent2.bot, {'port': serverPortViewer, 'firstPerson': True})
# browser = puppeteer.launch({'executablePath': 'C:/Program Files/Google/Chrome/Application/chrome.exe'})
# page = browser.newPage()
# page.goto(f'http://localhost:3100')


### Visualization code

In [ ]:

def outout_critic_images(agent2, three_d_info):
# def outout_critic_images(agent2, three_d_info, output_root): # uncomment it if you use Mineflayer Viewer.
    # width:x, depth:z, height:y
    # agent2.bot.chat(f'/gamemode creative')
    import time
    width = three_d_info["width"]
    depth = three_d_info["depth"]
    pivot_position = (agent2.pivot_pos[0]+three_d_info["width"]/2, agent2.pivot_pos[1]+three_d_info["height"]/2, agent2.pivot_pos[2]+three_d_info["depth"]/2)
    pivot_position_vec3 = vec3(agent2.pivot_pos[0]+three_d_info["width"]/2, agent2.pivot_pos[1]+three_d_info["height"]/2, agent2.pivot_pos[2]+three_d_info["depth"]/2)

    # you can edit these positions based on your requirements,
    # the observing agent will be teleported to these positions to look around the building.
    north_obs_pos = f"{pivot_position[0]} {pivot_position[1]+2} {pivot_position[2]+depth/2+8}"
    south_obs_pos = f"{pivot_position[0]} {pivot_position[1]+2} {pivot_position[2]-depth/2-8}"
    west_obs_pos = f"{pivot_position[0]+width/2+8} {pivot_position[1]+2} {pivot_position[2]}"
    east_obs_pos = f"{pivot_position[0]-width/2-8} {pivot_position[1]+2} {pivot_position[2]}"

    # the stand for observing agent. TODO: need to debug.
    north_obs_pos_stand = f"{pivot_position[0]} {pivot_position[1]} {pivot_position[2]+depth/2+8}"
    south_obs_pos_stand = f"{pivot_position[0]} {pivot_position[1]} {pivot_position[2]-depth/2-8}"
    west_obs_pos_stand = f"{pivot_position[0]+width/2+8} {pivot_position[1]} {pivot_position[2]}"
    east_obs_pos_stand = f"{pivot_position[0]-width/2-8} {pivot_position[1]} {pivot_position[2]}"

    time.sleep(1)
    agent2.bot.chat(f'/setblock {west_obs_pos_stand} minecraft:barrier')
    agent2.bot.chat(f'/tp bot {west_obs_pos} 90 0')
    # time.sleep(0.5)
    # agent2.bot.look(math.pi/2, 0, True) # yaw, pitch
    # agent2.bot.lookAt(pivot_position_vec3, True)
    time.sleep(1)
    # page.screenshot({'path': f'{output_root}screenshot_facing_west.png'}) # uncomment it if you use Mineflayer Viewer.
    agent2.bot.chat(f'/setblock {west_obs_pos_stand} minecraft:air')

    # time.sleep(2)

    agent2.bot.chat(f'/setblock {north_obs_pos_stand} minecraft:barrier')
    agent2.bot.chat(f'/tp bot {north_obs_pos} 180 0')
    # time.sleep(0.5)
    # agent2.bot.look(math.pi, 0, True) # yaw, pitch
    # agent2.bot.lookAt(pivot_position_vec3, True)
    time.sleep(1)
    # page.screenshot({'path': f'{output_root}screenshot_facing_north.png'}) # uncomment it if you use Mineflayer Viewer.
    agent2.bot.chat(f'/setblock {north_obs_pos_stand} minecraft:air')



    agent2.bot.chat(f'/setblock {east_obs_pos_stand} minecraft:barrier')
    # agent2.bot.chat(f'/tp bot {east_obs_pos}')
    agent2.bot.chat(f'/tp bot {east_obs_pos} -90 0')
    # time.sleep(0.5)
    # agent2.bot.look(math.pi/2, 0, True) # yaw, pitch
    # agent2.bot.lookAt(pivot_position_vec3, True)
    time.sleep(1)
    # page.screenshot({'path': f'{output_root}screenshot_facing_east.png'}) # uncomment it if you use Mineflayer Viewer.
    agent2.bot.chat(f'/setblock {east_obs_pos_stand} minecraft:air')

    # time.sleep(2)

    agent2.bot.chat(f'/setblock {south_obs_pos_stand} minecraft:barrier')
    agent2.bot.chat(f'/tp bot {south_obs_pos} 0 0')
    # time.sleep(0.5)
    # agent2.bot.look(0, 0, True) # yaw, pitch
    # agent2.bot.lookAt(pivot_position_vec3, True)
    time.sleep(1)
    # page.screenshot({'path': f'{output_root}screenshot_facing_south.png'}) # uncomment it if you use Mineflayer Viewer.
    agent2.bot.chat(f'/setblock {south_obs_pos_stand} minecraft:air')




# Main code for evaluation

In [ ]:
import json
import os
import time
from tqdm import tqdm
agent1.bot.chat(f'/gamemode spectator')

###
# input_root: the root path for generated buildings/assets.
# raw_info_root: architectures.json provided by our dataset repo (or the root path for new ground-truth architectures.)


input_root = "./_output/proprietary/" # specify based on your machine.
raw_info_root = "architectures.json" # specify based on your machine.
output_root = os.path.join(input_root, "tested_archs.json")

# specify based on your agents to be evaluated. 
# we recommend that the number of architectures built by agents to be tested should not exceed 2,000.
test_models = ['claude-3-5-sonnet', 'claude-3-7-sonnet-latest', 'gemini-1.5-flash-latest', 'gemini-1.5-pro', 'gemini-2.0-flash', 'gpt-4o', 'gpt-4o-mini']
agent1.init_env()

tested_archs = []
error_archs = []
error_reason = []

tasks = os.listdir(input_root)
for task in tqdm(tasks):
    if task.endswith("json"):
        continue
    error_data_path = os.path.join(input_root, task, 'errors.json')
    input_dir = os.path.join(input_root, task, 'llm_response_blueprint')

    test_archs = os.listdir(input_dir)
    with open(error_data_path, 'r') as f1:
        error_data_raw = json.load(f1)
    for item in error_data_raw:
        error_archs.append("%s$%s$%s" % (item["task"], item["architecture"], item["model"]))
    print(error_archs)

    for test_arch in tqdm(test_archs):
        with open(os.path.join(input_dir, test_arch), 'r') as f1:
            blueprint_data = json.load(f1)
        test_arch_name = test_arch.split('.json')[0]
        for k, v in blueprint_data.items():
            arch_indicator = f"{task}${test_arch}${k}" # Task_creativity$decoration_asset_118$gpt-4o-mini
            if arch_indicator in error_archs:
                continue
            else:
                tested_archs.append(arch_indicator)
            blueprint = v
            # clean the previous architecture. 
            # you can edit them based on your requirements.
            # Due to the limitation of block amount of this high-level command, adjustments should be made according to the actual condition
            agent1.bot.chat("/fill 1 1 1 40 20 40 minecraft:air")
            agent1.bot.chat("/fill 41 1 1 80 20 40 minecraft:air")
            agent1.bot.chat("/fill 1 1 41 40 20 80 minecraft:air")
            agent1.bot.chat("/fill 41 1 41 80 20 80 minecraft:air")
            agent1.bot.chat("/fill 1 1 1 25 50 25 minecraft:air")

            with open(raw_info_root, 'r') as f2:  # or your new ground-truth buildings, e.g., os.path.join(raw_info_root, test_arch)
                raw_info = json.load(f2)
            # print(raw_info['3d_info'])
            block_materials = raw_info["block_materials"]
            reverse_block_materials = {index+1: value for index, value in enumerate(block_materials)}
            reverse_block_materials.update({-1: 'air'})
            try:
                agent1.build_by_blueprint(blueprint, reverse_block_materials)
                three_d_info = {"height": len(blueprint), "depth": len(blueprint[0]), "width": len(blueprint[0][0])}
                agent1.bot.chat("/kill @e[type=item]") # clear the drop
            except Exception as e:
                # print(e)
                if e not in error_reason:
                    error_reason.append(e)

            outout_critic_images(agent2=agent2, three_d_info=three_d_info) # you can edit it based on your visualization tool.

print({item:"" for item in error_reason})
with open(output_root, 'w') as f1:
    json.dump(tested_archs, f1, indent=4)

### block matching
You can add this template code to the above main code according to actual needs. We will release a refined version later.

In [ ]:
import json
def v3(positions):
    return vec3(positions[0], positions[1], positions[2])

def block_matching(test_arch, pivot_position):
    with open('architecture.json', 'r') as f1:
        architecutes = json.load(f1)
    data = architecutes[test_arch]
    width = data['3d_info']['width'] # x axis
    height = data['3d_info']['height']  # y axis
    depth = data['3d_info']['depth']  # z axis
    blueprint = data['blueprint']
    # print(blueprint)
    block_amount = data['block_amount']
    correct_count = 0

    for h in range(height):
        for d in range(depth):
            for w in range(width):
                temp_position = (pivot_position[0] + w, pivot_position[1] + h, pivot_position[2] + d)
                blockat_data = agent2.bot.blockAt(v3(temp_position))
                if blockat_data.name != 'air' and blockat_data.name == blueprint[h][d][w]:
                    correct_count += 1
    # print(correct_count)
    # print(float(correct_count/block_amount))
    return float(correct_count/block_amount)


matching_rate = block_matching(test_arch='apple', pivot_position=(1,1,1))